# Imports

In [1]:
#!pip install numpy-stl

In [2]:
#!pip install -U scikit-learn


In [3]:
#!pip install vtkplotlib

In [4]:
from Transformation import Transformation
import numpy as np
import math
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
import networkx as nx

# Input

In [5]:
transformation = Transformation()

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [6]:
print(graph._node)

{(75.0252, -0.66553295, 3.8): {'index_triangle': {0, 6820, 164, 165, 6821, 6822}}, (75.01418, -0.49935842, 16.7): {'index_triangle': {0, 1, 165, 370, 371, 372}}, (75.0, 0.0, 3.8): {'index_triangle': {0, 1, 2, 6658, 6659, 11415, 11416, 11417, 6822, 6823, 6824, 6825, 6826, 6827, 6828, 6829, 6830, 6831, 6832, 6833, 6834, 6835, 6836, 6837, 6838, 6839, 6840, 6841, 6842, 6843, 6844, 6845, 6846, 6847, 6848, 6849, 6850, 6851, 6852, 6853, 6854, 6855, 6856, 6857, 6858, 6859, 6860, 6861, 6862, 6863, 6864, 6865, 6866, 6867, 6868, 6869, 6870, 6871, 6872, 6873, 6874, 6875, 6876, 6877, 6878, 6495}}, (75.00158, 0.16653232, 16.7): {'index_triangle': {1, 2, 3, 372, 373, 374}}, (75.0252, 0.66553295, 3.8): {'index_triangle': {2, 6659, 3, 4, 6660}}, (75.03937, 0.8314692, 16.7): {'index_triangle': {3, 4, 5, 374, 375, 376}}, (75.10067, 1.3272538, 3.8): {'index_triangle': {6660, 6661, 4, 5, 6, 6662}}, (75.12734, 1.4916434, 16.7): {'index_triangle': {5, 6, 7, 376, 377, 378}}, (75.22596, 1.9813722, 3.8): {'inde

# Point Sampler

### DevConv

In [7]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [8]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([1, 2, 3])  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):
            # print(index_current_node, " | ", current_node, " | ", dict_neigh)
            neigh_distances = np.zeros(shape=len(dict_neigh))
            for index_neigh, neigh_node in enumerate(dict_neigh.keys()):
                neigh_distances[index_neigh] = np.linalg.norm(np.multiply(self.W_theta, np.array(current_node) - np.array(neigh_node))) # norm(W_theta * (xi - xj))
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                                   # W_phi * max(distance)    

        # return if no previous inclusion score
        if len(previous_inclusion_score)==0:
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        # mean the rows if more than one column
        if list_inc_score.shape[1]!=1:
            list_inc_score = np.mean(list_inc_score, axis=1)

        # array of array to array
        if len(list_inc_score.shape)==2:
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [9]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[2.51159927 2.51224609 2.51160553 ... 0.78904169 2.26419359 0.78917508]
[10.18330284 10.18592537 10.18332825 ...  3.19917696  9.18019421
  3.1997178 ]
[0.99999987 0.99999987 0.99999987 ... 0.99324512 0.9999994  0.99325078]
(5999,)


### Multinomial Sampling

In [10]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[1 0 0 ... 0 0 0]


In [11]:
target_number_point = min(len(graph._node), 50)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]
list_k_nodes = [tuple(e) for e in list_k_nodes]
print(list_k_nodes)

[(78.60937, -7.138877, -9.670343), (75.72974, -8.610001, -3.1429949), (77.400055, 6.0434117, 3.6742187), (75.0, -8.798424, 90.166534), (81.3266, 8.447791, 3.468437), (83.196976, -7.138877, -7.4821677), (81.97276, -7.138877, -8.27718), (77.094795, -7.5075808, -9.426022), (87.10563, 6.2811418, 89.995255), (75.02465, 0.66553295, 3.79476), (75.09574, -1.3272538, 81.23111), (75.27572, 3.8609922, 2.9514337), (82.98924, -8.798424, -0.27072048), (75.46289, 3.2518802, 3.3832114), (82.76524, -8.798424, -0.68326616), (75.0, -8.610001, 88.18124), (51.0, 6.1633806, 83.71886), (75.0, -8.237194, 86.90345), (78.60937, 7.138877, 94.67034), (78.92292, -7.328478, 3.5944085), (76.20523, -4.4479885, 3.7368364), (75.22565, -1.9813722, 3.7881742), (88.219635, 6.7292814, 87.08576), (83.341095, -8.760632, -1.0157346), (81.06128, -8.760632, -3.6850574), (75.06263, -2.6241417, 81.595436), (78.00863, 6.7292814, 95.35448), (75.023026, 0.66553295, 3.7897491), (89.88141, 6.2811418, 82.7641), (78.37219, 4.7321377, 97

# KNN

In [12]:
XYZ = [list(e) for e in list_k_nodes]
print(XYZ)

[[78.60937, -7.138877, -9.670343], [75.72974, -8.610001, -3.1429949], [77.400055, 6.0434117, 3.6742187], [75.0, -8.798424, 90.166534], [81.3266, 8.447791, 3.468437], [83.196976, -7.138877, -7.4821677], [81.97276, -7.138877, -8.27718], [77.094795, -7.5075808, -9.426022], [87.10563, 6.2811418, 89.995255], [75.02465, 0.66553295, 3.79476], [75.09574, -1.3272538, 81.23111], [75.27572, 3.8609922, 2.9514337], [82.98924, -8.798424, -0.27072048], [75.46289, 3.2518802, 3.3832114], [82.76524, -8.798424, -0.68326616], [75.0, -8.610001, 88.18124], [51.0, 6.1633806, 83.71886], [75.0, -8.237194, 86.90345], [78.60937, 7.138877, 94.67034], [78.92292, -7.328478, 3.5944085], [76.20523, -4.4479885, 3.7368364], [75.22565, -1.9813722, 3.7881742], [88.219635, 6.7292814, 87.08576], [83.341095, -8.760632, -1.0157346], [81.06128, -8.760632, -3.6850574], [75.06263, -2.6241417, 81.595436], [78.00863, 6.7292814, 95.35448], [75.023026, 0.66553295, 3.7897491], [89.88141, 6.2811418, 82.7641], [78.37219, 4.7321377, 97

In [13]:
def connect_extended_graph(XYZ, number_neigh=3):
    is_graph_connected = False
    while not is_graph_connected:
        # Create the nearest neighbors object
        _, indices = NearestNeighbors(n_neighbors=number_neigh).fit(XYZ).kneighbors(XYZ)
        # Create the graph from the nearest points
        extended_graph = nx.Graph()
        for index_poly, poly in enumerate(indices):
            for index_current_node in range(len(poly)):
                current_node = list_k_nodes[poly[index_current_node]]
                for index_other_node in range(index_current_node+1, len(poly)):
                    edge = current_node, list_k_nodes[poly[index_other_node]]
                    extended_graph.add_edge(*edge)
                    # if attribute do not exists
                    if len(extended_graph.nodes[current_node])==0:
                        extended_graph.nodes[current_node]['index_poly'] = set()
                    extended_graph.nodes[current_node]['index_poly'].add(index_poly)
        is_graph_connected = nx.is_connected(extended_graph)
        number_neigh += 1
    print("Number of neighbors : ", number_neigh-1)
    return extended_graph

In [14]:
extended_graph = connect_extended_graph(XYZ)

Number of neighbors :  23


In [15]:
transformation.print_graph_properties(graph=extended_graph, display_graph=True, display_labels=False)

Number of nodes: 50
Number of edges: 758


# Edge Predictor

In [16]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
print(inclusion_score.shape)

(50, 64)


In [17]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""
from numpy import mean


f1i = np.mean(inclusion_score)
f1j = np.mean(inclusion_score)
wq = np.random.randint(100, size=(64,1))
wk = np.random.randint(100, size=(1,64))

f1i_wq = f1i*wq
f1j_wk = f1j*wk

wq_wk = int(np.matmul(f1j_wk, f1i_wq)) 

expo = np.exp(wq_wk)



C:\Users\debor\AppData\Local\Temp\ipykernel_15460\73126498.py:21: RuntimeWarning: overflow encountered in exp
  expo = np.exp(wq_wk)


### Sparse Attention

# Face Candidates

# Face Classifier

### TriConv

# Simplified Mesh

# Fonctions Loss

Récupération des points de l'espace de départ P

In [18]:
keys_input = list(graph._node.keys())
print(keys_input)
len(keys_input)  #return 5999

[(75.0252, -0.66553295, 3.8), (75.01418, -0.49935842, 16.7), (75.0, 0.0, 3.8), (75.00158, 0.16653232, 16.7), (75.0252, 0.66553295, 3.8), (75.03937, 0.8314692, 16.7), (75.10067, 1.3272538, 3.8), (75.12734, 1.4916434, 16.7), (75.22596, 1.9813722, 3.8), (75.26499, 2.1432736, 16.7), (75.40037, 2.6241417, 3.8), (75.45153, 2.7826276, 16.7), (75.62288, 3.2518802, 3.8), (75.68588, 3.4060426, 16.7), (75.892235, 3.8609922, 3.8), (75.96672, 4.0099483, 16.7), (76.20689, 4.4479885, 3.8), (76.29242, 4.590885, 16.7), (76.565025, 5.0095077, 3.8), (76.661125, 5.145526, 16.7), (76.96461, 5.5423326, 3.8), (77.07072, 5.670694, 16.7), (77.40334, 6.0434117, 3.8), (77.51886, 6.1633806, 16.7), (77.87872, 6.5098743, 3.8), (78.002975, 6.620764, 16.7), (78.388016, 6.9390492, 3.8), (78.5203, 7.0402236, 16.7), (78.92831, 7.328478, 3.8), (79.06786, 7.4193583, 16.7), (79.4965, 7.6759295, 3.8), (79.64253, 7.755995, 16.7), (80.08935, 7.9794145, 3.8), (80.24101, 8.048206, 16.7), (80.703445, 8.237194, 3.8), (80.859886, 

5999

Récupération des points de l'espace de départ Ps

In [28]:
keys_output = list(extended_graph)
print(keys_output)
len(keys_output)  #return: 50

[(78.60937, -7.138877, -9.670343), (77.094795, -7.5075808, -9.426022), (81.97276, -7.138877, -8.27718), (83.196976, -7.138877, -7.4821677), (81.06128, -8.760632, -3.6850574), (75.72974, -8.610001, -3.1429949), (84.9552, -7.8332825, -4.261562), (75.79615, -7.9794145, -1.2266885), (79.43241, -8.237194, 0.21070446), (83.341095, -8.760632, -1.0157346), (82.76524, -8.798424, -0.68326616), (82.98924, -8.798424, -0.27072048), (88.121735, -4.7321377, -5.7334995), (77.9193, -7.328478, 1.171451), (78.018745, -6.9390492, 2.261874), (78.92292, -7.328478, 3.5944085), (76.20523, -4.4479885, 3.7368364), (75.22565, -1.9813722, 3.7881742), (75.00263, -0.66553295, 3.7749352), (75.023026, 0.66553295, 3.7897491), (75.02465, 0.66553295, 3.79476), (75.46289, 3.2518802, 3.3832114), (75.27572, 3.8609922, 2.9514337), (77.400055, 6.0434117, 3.6742187), (81.3266, 8.447791, 3.468437), (82.485054, 8.760632, -2.2612798), (91.55599, 4.1574683, 3.8), (75.0, -8.798424, 90.166534), (75.0, -8.610001, 88.18124), (75.0, -

50

Création de la liste normes entre chaque point

In [20]:
import numpy as np

def compare_and_norm(keys_x, keys_y):
    normes_carre = []

    for x in keys_x:
        # Convertir les tuples en arrays NumPy pour le calcul de la norme
        keys_x_np = np.array(x)
        # Calculer la norme euclidienne au carré pour chaque point y dans keys_y
        distances_carre = [np.linalg.norm(keys_x_np - np.array(y))**2 for y in keys_y]
        # Ajouter la distance minimale au carré à la liste
        norme_carre = min(distances_carre)
        normes_carre.append(norme_carre)

    return normes_carre

def min_pour_y_list(keys_x, keys_y):
    # Calculer le min pour chaque point y dans keys_y
    return min(compare_and_norm(keys_x, keys_y))

def min_pour_x_list(keys_x, keys_y):
    # Calculer le min pour chaque point x dans keys_x
    return min(compare_and_norm(keys_x, keys_y))


In [21]:
compare_and_norm(keys_input, keys_output)

[0.001137542788157156,
 167.08506628994746,
 0.4435691908329318,
 166.79476068300664,
 2.775896368913144e-05,
 166.57298762039227,
 0.44368090602234034,
 167.2382560927881,
 1.7719858585078327,
 168.7867265695386,
 0.5716774739675579,
 171.20956408334678,
 0.19930900297531995,
 174.49286373907762,
 0.729066654286644,
 175.86041060187654,
 1.931702629877293,
 173.00767768474543,
 1.782053230037718,
 171.0232102516611,
 0.45651581036521804,
 169.9183565739304,
 0.01583174758352257,
 169.6994851859572,
 0.4625315832274879,
 170.36784985772465,
 1.794054134777312,
 171.91957941840883,
 4.002768995583267,
 174.3458047840477,
 4.054973320707234,
 177.63263699000618,
 1.860101448288333,
 176.41245039315345,
 0.5426056129182513,
 175.3156505104207,
 0.1100094080943883,
 175.10486328514526,
 0.5647997472296993,
 175.7813004863865,
 1.9043680722621161,
 177.34107695701914,
 4.121049922017846,
 179.77523100814233,
 7.202111777940047,
 183.0698477137048,
 11.129972720347325,
 187.20602734334898,
 

Chamfer distance

In [31]:
def d_P_Ps(keys_x, keys_y, p_y):
    min_for_x = [p_y * min_pour_x_list([keys_x[i]], keys_y) for i in range(len(keys_x))]
    
    min_for_y = [p_y * min_pour_y_list([keys_y[j]], keys_x) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(p_y * min_for_x)
    sum_for_x = np.sum(p_y * min_for_y)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance



In [32]:
# Utilisation de la fonction avec les clés d'entrée (keys_input), les clés de sortie (keys_output)
# et les probabilités normalisées (normalized_inclusion_score)
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Probabilistic Chamfer Distance:", prob_chamfer_dist)

Probabilistic Chamfer Distance: 64.84988623431525


## Probabilistic Surfaces Distance

In [24]:
def calculate_distance_between_surfaces(b, b_hat):
    """
    calculates the distance between probabilistic surfaces using their barycenters.
    
    Args:
    - b: Barycenters of the ground truth surface
    - b_hat: Barycenters of generated triangles with respective probabilities

    Returns:
    - Distance between probabilistic surfaces
    """
    distances = np.linalg.norm(b_hat[:, :3][:, np.newaxis, :] - b, axis=2)**2
    weighted_distances = b_hat[:, 3] * np.min(distances, axis=1)
    total_distance = np.sum(weighted_distances)
    return total_distance



In [25]:
# Exemple d'utilisation 
b_ground_truth = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
b_generated = np.array([[0.5, 1.5, 2.5, 0.8], [4.5, 5.5, 6.5, 0.6], [6.0, 8.0, 8.0, 0.7]])

distance_result = calculate_distance_between_surfaces(b_ground_truth, b_generated)

print("Distance entre surfaces probabilistes :", distance_result)

Distance entre surfaces probabilistes : 2.45


## Triangle Collision Loss

In [26]:
def triangle_collision_loss(generated_triangles, probabilities):
    """
    Calculate triangle collision loss to penalize triangles that penetrate each other.

    Parameters:
    - generated_triangles (numpy.ndarray): Array representing the generated triangles.
    - probabilities (numpy.ndarray): Array of probabilities for each generated triangle.

    Returns:
    - float: Triangle collision loss.
    """
    num_triangles = len(generated_triangles)

    # Create a 3D array to represent all combinations of triangle pairs
    triangle_pairs_i, triangle_pairs_j = np.meshgrid(range(num_triangles), range(num_triangles), indexing='ij')

    # Filter out pairs where i is not equal to j
    valid_pairs_mask = triangle_pairs_i != triangle_pairs_j

    # Extract triangles and probabilities for each pair
    triangles_i = generated_triangles[triangle_pairs_i[valid_pairs_mask]]
    triangles_j = generated_triangles[triangle_pairs_j[valid_pairs_mask]]
    probs_i = probabilities[triangle_pairs_i[valid_pairs_mask]]

    # Calculate the plane normals and distances for all triangles
    plane_normals = np.cross(triangles_i[:, 1] - triangles_i[:, 0], triangles_i[:, 2] - triangles_i[:, 0])

    # Calculate the edge vectors for all edges of triangles_j
    edge_vectors = triangles_j[:, [1, 2, 0], :] - triangles_j[:, [0, 1, 2], :]

    # Calculate the dot product of edge vectors and plane normals
    dot_products = np.sum(np.cross(edge_vectors, plane_normals[:, np.newaxis, :]) * triangles_i[:, 0], axis=2)

    # Count the number of faces penetrated for each triangle
    faces_penetrated = np.sum(dot_products < 0, axis=1)

    # Calculate the loss for each triangle
    loss_per_triangle = probs_i * faces_penetrated

    # Sum up the losses and normalize by the number of triangles
    triangle_losses = np.sum(loss_per_triangle) / num_triangles

    return triangle_losses

